In [ ]:
# Generators are iterable
def gen():
    yield 1
    yield 2
    yield 3
[i for i in gen()]

In [ ]:
# Generator returning generator
def gen1():
    yield 1

def gen2():
    yield 2

def gen():
    yield from gen1()
    yield from gen2()

[i for i in gen()]

In [ ]:
# Generator comprehension

# Could do this with a list comprehension and take the first element, but that
# would create the entire list just to get the first value.
x = (i for i in range(1000) if i**2 > 1000)
next(x)

In [ ]:
# Stateful Coroutine
def coro():
    print('starting')
    x = yield 1
    print(f'x = {x}')
    y = yield 2
    print(f'y = {y}')

c = coro()
print(next(c))

# Sends this value back to the yield statement and continues on to the next yield.
print(c.send(3))

# Calling next here would continue the function to print y and then raise StopIteration
# since there is no yield statement left.

In [ ]:
# Mutability - can you change the value pointed to or just the pointer.
# E.g. ints look mutable because we mutate `n` but really we point to a different value.
n = 1
print(id(n))
n += 1
print(id(n))


In [ ]:
# (Im)Mutability doesn't effect sub objects.
# Tuples are immutable, but we can still mutate internals.
x = [], 1
print(x, id(x))
x[0].append(1)
print(x, id(x))

In [ ]:
# Mutability means that a change in 1 location can affect other places.
ll = [[]] * 5  # Repeats the same empty list (by ref) for all elements
ll[0].append(1)
ll

In [ ]:
# Proper display
class User:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def __repr__(self):
        # Show the construction with the repr of each param.
        return f'{self.__class__.__name__}({self.name!r})'


user = User('Alice')
user

In [ ]:
# Equality:
# Generally Python favors calling the subclass for operators, not just for __eq__.
class A:
    def __init__(self, x):
        self.x = x
    
    def __eq__(self, other):
        print(f'A.__eq__({self!r}, {other!r}')
        return isinstance(other, A) and self.x == other.x

class B(A):
    def __eq__(self, other):
        print(f'B.__eq__({self!r}, {other!r}')
        return isinstance(other, B) and self.x == other.x
    
class C:
    def __init__(self, x):
        self.x = x
    
    def __eq__(self, other):
        print(f'C.__eq__({self!r}, {other!r}')
        return isinstance(other, C) and self.x == other.x

a = A(1)
b = B(1)
c = C(1)
print(a == b, b == a)
print(a == c, c == a)
print(b == c, c == b)

In [ ]:
# Comparison - different types shouldn't imply a >=< relationship.
# For equality this is simply false. For comparison this is an erro.
# Equality seems to favor the subclass.
class A:
    def __init__(self, x):
        self.x = x
    
    def __gt__(self, other):
        print(f'A.__gt__({self!r}, {other!r}')
        if not isinstance(other, A):
            # Signals to python to flip: self > other into (other <)
            return NotImplemented
        return self.x > other.x

class Epsilon:
    def __lt__(self, other):
        print(f'Epsilon.__lt__({self!r}, {other!r}')
        return True

e = Epsilon()
a = A(1)
print(e < a)
print(a > e)


In [ ]:
# Callable classes can be decorators.
class Memoized:
    def __init__(self, f):
        self.f = f
        self.cache = {}
    
    def __call__(self, *args, **kwargs):
        token = args + tuple(kwargs.items())
        if token not in self.cache:
            self.cache[token] = self.f(*args, **kwargs)
        return  self.cache[token]

# Calls init on fib.
@Memoized
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)

print(fib(10))
fib.cache

In [ ]:
# Build our own data frame with getitem
class DataFrame:
    """
    Matrix where each col is identified by a string and contains a list of values.
    {'col_name' : [values]}
    """
    def __init__(self, values):
        self.values = values
    def __getitem__(self, arg):
        if isinstance(arg, str):
            return Filter(arg, self.values) 
        if isinstance(arg, list):
            return [val for val, include in zip(self.values, arg) if include]

class Filter:
    def __init__(self, key, values):
        self.key = key
        self.values = values
    def __gt__(self, other):
        return [val[self.key] > other for val in self.values]

df = DataFrame([{'x': i, 'y': i**2} for i in range(5)])
df[df['y'] > 5]

In [ ]:
# Iterables
class A:
    def __init__(self, x):
        self.x = x
    def __iter__(self):
        for i in range(self.x):
            yield self.x

a = A(3)
[i for i in a]

In [ ]:
# Method resolution order - Diamond Inheritance
class A:
    x = 1
class B(A):
    x = 2
class C(A):
    x = 3
class D(B, C):
    pass

d = D()
print(d.x)
D.__mro__

In [ ]:
# Super calls to all parents. Iterates over self.__class__.__mro__.
class A:
    def f(self):
        print('A')
class B(A):
    def f(self):
        print('B')
        super().f()
class C(A):
    def f(self):
        print('C')
        super().f()
class D(B, C):
    def f(self):
        print('D')
        super().f()

d = D()
d.f()

In [7]:
# Attribute priority. Note similarity to namespace.
class A:
    x = 'class'  # Class Attribute
    def __init__(self):
        self.x = 'instance'  # Object/Instance Attribute
    def __getattr__(self, key):
        # Dynamic Attribute - there's also set & del.
        return key

a = A()
b = A()

delattr(a, 'x')  # Delete object attribute. Only effects `a`.
print(a.x, b.x)  # Lookup now falls back to class attribute.

delattr(b, 'x')
b.x = 'newval'  # Updates object attribute, not class attribute.
print(a.x, b.x)

delattr(b, 'x')  # Delete the object attribute.
delattr(A, 'x')  # Delete class attribute.
print(a.x, b.x)  # All instances now lack the class attribute.

class instance
class newval
x x


In [8]:
# Descriptors take precedence over dynamic attributes of the class.
class D:
    def __get__(self, instance, cls):
        print(f'get {instance=} {cls=}')
        return 42
    def __set__(self, instance, value):
        print(f"setting {instance}'s value to {value!r}")
    def __delete__(self, instance):
        print(f"deleting {instance}'s value")

class A:
    # Descriptors must be set at the class.
    d = D()

a = A()
print(a.d)
a.d = 1  # Calls to set, doesn't create a new field.
del a.d
print(a.d)

get instance=<__main__.A object at 0x7fabf5260670> cls=<class '__main__.A'>
42
setting <__main__.A object at 0x7fabf5260670>'s value to 1
deleting <__main__.A object at 0x7fabf5260670>'s value
get instance=<__main__.A object at 0x7fabf5260670> cls=<class '__main__.A'>
42
